In [1]:
import os
os.chdir('../')
%pwd

'c:\\Users\\LONG\\Computer Science\\Artificial Intelligence\\Projects\\End-To-End-Chicken-Disease-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories(
            [Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)]
        )

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=config.root_dir,
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        return prepare_callbacks_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config=config
        
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir)